# Assignment 6

In this assignment, you'll analyze a collection of data sets from the [San Francisco Open Data Portal](http://data.sfgov.org/) and [Zillow](https://www.zillow.com/). The data sets have been stored in the SQLite database `sf_data.sqlite`, which you can [download here](http://anson.ucdavis.edu/~nulle/sf_data.sqlite). The database contains the following tables:

Table                   | Description
----------------------- | -----------
`crime`                 | Crime reports dating back to 2010.
`mobile_food_locations` | List of all locations where mobile food vendors sell.
`mobile_food_permits`   | List of all mobile food vendor permits. More details [here](https://data.sfgov.org/api/views/rqzj-sfat/files/8g2f5RV4PEk0_b24iJEtgEet9gnh_eA27GlqoOjjK4k?download=true&filename=DPW_DataDictionary_Mobile-Food-Facility-Permit.pdf).
`mobile_food_schedule`  | Schedules for mobile food vendors.
`noise`                 | Noise complaints dating back to August 2015.
`parking`               | List of all parking lots.
`parks`                 | List of all parks.
`schools`               | List of all schools.
`zillow`                | Zillow rent and housing statistics dating back to 1996. More details [here](https://www.zillow.com/research/data/).

The `mobile_food_` tables are explicitly connected through the `locationid` and `permit` columns. The other tables are not connected, but you may be able to connect them using dates, latitude/longitude, or postal codes.

Shapefiles for US postal codes are available [here](https://www.census.gov/geo/maps-data/data/cbf/cbf_zcta.html). These may be useful for converting latitude/longitude to postal codes.

Shapefiles for San Francisco Neighborhoods are available [here](https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4).

In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
db = sql.connect("sf_data.sqlite")
pd.read_sql("SELECT * FROM sqlite_master", db)

,type,name,tbl_name,rootpage,sql
0,table,crime,crime,2,"CREATE TABLE ""crime"" (\n""IncidntNum"" INTEGER,\..."
1,table,noise,noise,35775,"CREATE TABLE ""noise"" (\n""CaseID"" INTEGER,\n ""..."
2,table,parking,parking,35921,"CREATE TABLE ""parking"" (\n""Owner"" TEXT,\n ""Ad..."
3,table,schools,schools,35944,"CREATE TABLE ""schools"" (\n""Name"" TEXT,\n ""Ent..."
4,table,parks,parks,35961,"CREATE TABLE ""parks"" (\n""Name"" TEXT,\n ""Type""..."
5,table,zillow,zillow,35967,"CREATE TABLE ""zillow"" (\n""RegionName"" INTEGER,..."
6,table,mobile_food_permits,mobile_food_permits,36050,"CREATE TABLE ""mobile_food_permits"" (\n""permit""..."
7,table,mobile_food_locations,mobile_food_locations,36060,"CREATE TABLE ""mobile_food_locations"" (\n""locat..."
8,table,mobile_food_schedule,mobile_food_schedule,36079,"CREATE TABLE ""mobile_food_schedule"" (\n""locati..."


__Exercise 1.1.__ Which mobile food vendor(s) sells at the most locations?

In [9]:
require = """ SELECT
                  L.locationid, 
                  M.Applicant,
                  M.permit, 
                  COUNT(DISTINCT locationid) AS counts 
            FROM 
                 mobile_food_permits AS M
            INNER JOIN mobile_food_schedule  AS L ON L.permit = M.permit 
            GROUP BY 
                 M.Applicant
            ORDER BY counts DESC"""
data1 = pd.read_sql(require,db)
data1.head()

,locationid,Applicant,permit,counts
0,934630,May Catering,17MFF-0110,58
1,934668,Anas Goodies Catering,17MFF-0111,37
2,437226,Natan's Catering,13MFF-0102,37
3,934485,Liang Bai Ping,17MFF-0090,33
4,765710,Park's Catering,16MFF-0051,23


In [10]:
db.close()

__Exercise 1.2.__ Ask and use the database to analyze 5 questions about San Francisco. For each question, write at least 150 words and support your answer with plots. Make a map for at least 2 of the 5 questions.

You should try to come up with some questions on your own, but these are examples of reasonable questions:

* Which parts of the city are the most and least expensive?
* Which parts of the city are the most dangerous (and at what times)?
* Are noise complaints and mobile food vendors related?
* What are the best times and places to find food trucks?
* Is there a relationship between housing prices and any of the other tables?

Please make sure to clearly state each of your questions in your submission.

In [2]:
import geopandas as gpd
import shapely.geometry as geom
# Make Jupyter to automatically display matplotlib plots.
%matplotlib inline
# Make plots larger.
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 12)
import pandas as pd
import numpy as np

In [3]:
db = sql.connect("sf_data.sqlite")

### 2.1 Which parts of the city are the most and least expensive?

I choose to use table zillow to do the analysis. There are 6 variables in this table.

- ZRI per Sq Ft ($): Median of the estimated monthly rent price of all homes, per square foot. This is calculated by taking the estimated rent price for a home and dividing it by the homes square footage.

- Median Sold Price Per Sq Ft ($): Median of sold prices divided by the square footage of a home.

- Price to Rent Ratio: This ratio is first calculated at the individual home level, where the estimated home value is divided by 12 times its estimated monthly rent price. The the median of all home-level price-to-rent ratios for a given region is then calculated.

- Sold in Past Year (Turnover) (%): The percentage of all homes in a given area that sold in the past 12 months.

In [16]:
Housing = pd.read_sql("SELECT * FROM zillow", db)
Housing.head()

,RegionName,Date,ZriPerSqft_AllHomes,MedianSoldPricePerSqft_AllHomes,PriceToRentRatio_AllHomes,Turnover_AllHomes
0,94109,2010-11-01 00:00:00,3.156,675.1913,19.14,6.0771
1,94110,2010-11-01 00:00:00,2.566,599.6785,18.10,5.4490
2,94122,2010-11-01 00:00:00,2.168,495.4432,17.99,2.4198
3,94080,2010-11-01 00:00:00,1.666,369.5538,18.31,3.8757
4,94112,2010-11-01 00:00:00,2.322,422.4538,14.44,3.1288


In [22]:
require1 = """SELECT 
                 RegionName, 
                 AVG(ZriPerSqft_AllHomes) AS avg_ZP, 
                 AVG(MedianSoldPricePerSqft_AllHomes) AS avg_MSP, 
                 AVG(PriceToRentRatio_AllHomes) AS avg_PRR,
                 AVG(Turnover_AllHomes) AS avg_T
            FROM 
                 zillow 
            GROUP BY RegionName 
            ORDER BY avg_MSP
"""
msp = pd.read_sql(require1,db)
msp.head()

,RegionName,avg_ZP,avg_MSP,avg_PRR,avg_T
0,94124,2.328160,297.008046,15.119605,4.810203
1,94134,2.395173,347.027455,17.030000,3.852362
2,94080,2.172000,375.629802,18.342105,4.413822
3,94132,2.370480,385.384982,19.757368,4.093720
4,94112,2.466640,416.744886,17.992895,3.745187


In [25]:
msp.idxmax(axis=0, skipna=True)

RegionName    22
avg_ZP        24
avg_MSP       25
avg_PRR       23
avg_T         25
dtype: int64

In [28]:
msp.iloc[23:26]

,RegionName,avg_ZP,avg_MSP,avg_PRR,avg_T
23,94123,3.764773,696.166788,26.086711,5.839732
24,94105,4.279173,798.257824,20.133816,6.928359
25,94104,NaN,1336.128581,NaN,14.831512


In [26]:
msp.idxmin(axis=0, skipna=True)

RegionName    2
avg_ZP        2
avg_MSP       0
avg_PRR       0
avg_T         7
dtype: int64

There are many different house with same region name, so I take the average of them by region name. In this way, I got three new variables

- avg_ZP, average of the ZRI per Sq Ft with same region name
- avg_MSP, average of Median Sold Price Per Sq Ft with same region name
- avg_PRR, average of Price to Rent Ratio with same region name
- avg_T, average of Turnover in the same region
#### Conclusions:
 1. Region 94124 has the lowest Median Sold Price Per F Sqt and theest  lowPrice to Rent Ratio
 2. Houses in region 94104 is the most expensive but its turnover is the highest and these houses are not for rent. So we can guess region 94104 is a valuable region.
 3. Region 94123 has the highest average of Price to Rent Ratio. It means that renting a house in this area may better than buying a house.
 4. Region 94105 has the most expensive rent price. From the region name 94104 and 94105, I guess these two area is closed.

### 2.2 Which parts of the city are the most dangerous (and at what times)?

The table crime is suitable to solve this problem.

In [4]:
crime = pd.read_sql("SELECT * FROM crime", db)
crime.head()

,IncidntNum,Category,Descript,DayOfWeek,Datetime,PdDistrict,Resolution,Address,Lon,Lat
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,2015-01-19 14:00:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,2015-01-27 19:00:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469


In [5]:
zips = gpd.read_file("cb_2015_us_zcta510_500k")

zips_sf = zips[zips.ZCTA5CE10.str.startswith("941")]
zips_sf = zips_sf[pd.to_numeric(zips.ZCTA5CE10) <= 94158]
zips_sf = zips_sf.drop(25170,axis=0)

D:\anaconda2\lib\site-packages\geopandas\geodataframe.py:376: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [6]:
lonlat_c = [geom.Point(lon, lat) for lon, lat in zip(crime.Lon, crime.Lat)]
crime.drop(["Lat", "Lon"], axis = 1)

crime= gpd.GeoDataFrame(crime, geometry = lonlat_c, crs = {'init' :'epsg:4326'})
crime.crs = crime.to_crs(zips.crs)
crime = crime.dropna()

In [10]:
def findzip(x):
    zipcode = []
    for pt in x.geometry:
        matched = zips_sf[zips_sf.geometry.contains(pt)]
        if matched.shape[0] == 0:
            zipcode.append(np.NaN)
        else:
            zipcode.append(matched.ZCTA5CE10.iloc[0])
    return zipcode

In [13]:
crime["zip"] = findzip(crime)
crime.head()

,IncidntNum,Category,Descript,DayOfWeek,Datetime,PdDistrict,Resolution,Address,Lon,Lat,geometry,zip
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,2015-01-19 14:00:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,POINT (-122.42158168137 37.7617007179518),94110
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,POINT (-122.414406029855 37.7841907151119),94109
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,POINT (-122.414406029855 37.7841907151119),94109
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,POINT (-122.414406029855 37.7841907151119),94109
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,2015-01-27 19:00:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,POINT (-122.431118543788 37.8004687042875),94123


In [ ]:
base = zips_sf.plot()
crime.plot(ax = base, marker = "o", color = "green")

### 2.3 Are noise complaints and mobile food vendors related?

This question need to use two data set noise and mobile_food_locations.

In [12]:
noise = pd.read_sql("SELECT * FROM noise", db)
noise.head()

,CaseID,Type,Address,Neighborhood,Datetime,Lat,Lon
0,5130305,other_excessive_noise,Not associated with a specific address,None,2015-09-25 00:29:33,NaN,NaN
1,6446138,amplified_sound_electronics,Intersection of CLINTON PARK and GUERRERO ST,Mission Dolores,2016-10-21 13:20:18,37.769148,-122.424475
2,5929789,other_excessive_noise,"1301 48TH AVE, SAN FRANCISCO, CA, 94122",Outer Sunset,2016-06-01 11:38:59,37.762047,-122.508365
3,5659943,other_excessive_noise,"736 LEAVENWORTH ST, SAN FRANCISCO, CA, 94109",Lower Nob Hill,2016-03-13 01:18:27,37.787983,-122.414943
4,5930704,other_excessive_noise,"1360 43RD AVE, SAN FRANCISCO, CA, 94122",Outer Sunset,2016-06-01 13:58:43,37.761385,-122.502232
